<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Merge_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importar bibliotecas necesarias
import pandas as pd

# Cargar los datasets desde archivos locales
assignments_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned_sin_cursos_reutilizados.csv"
exams_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/examenes/examenes_consolidados.csv"
access_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado_cleaned.csv"
forums_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/foros/foros_consolidado.csv"

assignments_df = pd.read_csv(assignments_path)
exams_df = pd.read_csv(exams_path)
access_df = pd.read_csv(access_path)
forums_df = pd.read_csv(forums_path)

# 1. Integrar "Exams" a "Assignments" basado en groupKey y email
merged_df = pd.merge(assignments_df, exams_df, how="left", on=["groupKey", "email"])

# 2. Integrar "Access" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, access_df, how="left", on=["curso", "groupKey", "email"])

# 3. Integrar "Forums" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, forums_df, how="left", on=["curso", "groupKey", "email"])

# 4. Exportar el dataset consolidado
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
merged_df.to_csv(output_path, index=False)

print(f"Dataset consolidado guardado en {output_path}")

Dataset consolidado guardado en /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv


**2. Limpieza de examenes**

In [6]:
import pandas as pd

# Definir rutas de entrada y salida
input_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_exam_cleaned_dataset.csv"

# Cargar el dataset
df = pd.read_csv(input_path)

# Contar estudiantes con datos de exámenes por curso
exam_distribution = df.groupby("groupKey")["total_assigned_exams"].count()
total_students_per_group = df.groupby("groupKey")["email"].count()

# Identificar los casos
courses_without_exams = exam_distribution[exam_distribution == 0].index  # Caso 1
courses_with_all_exams = exam_distribution[exam_distribution == total_students_per_group].index  # Caso 2
courses_some_missing_exams = exam_distribution[
    (exam_distribution / total_students_per_group >= 0.95) & (exam_distribution / total_students_per_group < 1)
].index  # Caso 3
courses_few_exams = exam_distribution[
    (exam_distribution / total_students_per_group <= 0.05) & (exam_distribution / total_students_per_group > 0)
].index  # Caso 4

# Imprimir conteo de cada caso
print("\nDistribución de Casos por Curso:")
print(f"Caso 1 - Cursos sin exámenes: {len(courses_without_exams)}")
print(f"Caso 2 - Cursos donde todos tienen exámenes: {len(courses_with_all_exams)}")
print(f"Caso 3 - Cursos con solo 5% alumnos sin exámenes: {len(courses_some_missing_exams)}")
print(f"Caso 4 - Cursos con solo 95% alumnos sin exámenes: {len(courses_few_exams)}")

# Crear la variable "no_exam_data" basado en total_assigned_exams
df["no_exam_data"] = df["total_assigned_exams"].isna().astype(int)

# Guardar el dataset limpio
df.to_csv(output_path, index=False)

# Confirmación de guardado
print(f"\nDataset limpio guardado en: {output_path}")



Distribución de Casos por Curso:
Caso 1 - Cursos sin exámenes: 23
Caso 2 - Cursos donde todos tienen exámenes: 130
Caso 3 - Cursos con solo 5% alumnos sin exámenes: 30
Caso 4 - Cursos con solo 95% alumnos sin exámenes: 1

Dataset limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_exam_cleaned_dataset.csv
